In [7]:
import os
import PyPDF2

#Function to Load pdf documents
def load_pdf_documents(directory):
    documents =[]
    file_names =[]
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path=os.path.join(directory,filename)
            with open(file_path,'rb') as file:
                reader=PyPDF2.PdfReader(file)
                text=''
                for page in range(len(reader.pages)):
                    text += reader.pages[page].extract_text()
                documents.append(text)
                file_names.append(filename)
    return documents, file_names

pdf_directory = r"Numerology" 
documents, file_names= load_pdf_documents(pdf_directory)


In [9]:
#Intialize the Vectorization and fit the documents
from sklearn.feature_extraction.text import TfidfVectorizer



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\suman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\suman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Vectorize the Query, Compute Cosine Similarity between the query vector and document vector, Fetch the index of most relevant document vector

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def retrieve_relevant_document(query,doc_vectors,vectorizer,documents,file_names):
    #Vectorizing the query
    query_vector = vectorizer.transform([query])
    #Compute Cosine Similarities 
    similarities =cosine_similarity(query_vector,doc_vectors).flatten()
    #Get the index of the most similar document
    most_similar_doc_index =np.argmax(similarities)
    return documents[most_similar_doc_index], file_names[most_similar_doc_index]

In [ ]:
#Load Pre trained model and tokenizer

from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer =GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')

def generate_response(query,context):
#Combine the context and query into a single input for the model
    input_text = f"{context} [SEP] {query}"
    input_ids =tokenizer.encode(input_text, return_tensors='pt')

#Generate the output
    output=model.generate(input_ids, max_length=50, num_return_sequences=1)
    response= tokenizer.decode(output[0], skip_special_tokens=True)
    return response

#RAG Pipeline

def rag_pipeline(query):
    #Step 1: Retrieving the most relevant document
    context, file_name = retrieve_relevant_document(query, doc_vectors,vectorizer, documents, file_names)
    print(f"Retrieved Document: {context[:200]}....\n")  # print first 200 characters of the document
    print(f"Document Source:{file_name}\n")

    # Step 2: Generate a response using the retrieved document as context
    response=generate_response(query,context)
    return response

query =input("Hi. I am your Travel assistant. How can I help you?")
response= rag_pipeline(query)
print(f"Generated Response:{response}")


